Read in all unformatted files. Each line is a data entry

In [1]:
import pandas as pd
import numpy as np

english_exclusive_pure=[i.strip() for i in list(open("../../data_unformatted/english_exclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
english_inclusive_pure=[i.strip() for i in list(open("../../data_unformatted/english_inclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
french_exclusive_pure=[i.strip() for i in list(open("../../data_unformatted/french_exclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
french_inclusive_pure=[i.strip() for i in list(open("../../data_unformatted/french_inclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]

if len(english_inclusive_pure) != len(english_exclusive_pure):
  raise Exception("uneven enlgish pairs")

if len(french_inclusive_pure) != len(french_exclusive_pure):
  raise Exception("uneven french pairs")

setup for zero-shot

In [2]:
both_exclusive_pure=english_exclusive_pure+french_exclusive_pure
both_inclusive_pure=english_inclusive_pure+french_inclusive_pure

# taskDescriptionOne="classify each of the sentences as inclusive or exclusive."
all_data=pd.DataFrame()

all_data["text"] = both_exclusive_pure+both_inclusive_pure
all_data["class"] = ['exclusive']*len(both_exclusive_pure)+['inclusive']*len(both_inclusive_pure)
print(all_data.head())

                                                text      class
0  African immigrants are only able to seek unqua...  exclusive
1           African Americans are always on welfare.  exclusive
2  He looks Middle Eastern so he is probably here...  exclusive
3  Oriental people built the transcontinental rai...  exclusive
4            Everything is fair, I do not see color.  exclusive


#### setup for few-shot  
- 4 random pairs
- 8 random pairs
- 4 inclusive and 4 exclusive. unrelated
- 8 inclusive and 8 exclusive. unrelated  
  
all are equal english and french.  
no sentence is reused. making datasets unique.

In [3]:
import random as ran

num_total_examples=len(english_inclusive_pure)+len(french_inclusive_pure)
both_example_range = range(num_total_examples)
single_example_range = range(len(english_exclusive_pure))

used_index = []

train_pairs_random_4=pd.DataFrame()
train_pairs_random_8=pd.DataFrame()
train_unrelated_random_8=pd.DataFrame()
train_unrelated_random_16=pd.DataFrame()

train_text_buffer=[]
train_class_buffer=[]

on_english=True
on_inclusive=True

train_pairs_random_4

In [4]:
train_text_buffer.clear()
train_class_buffer.clear()

four_ran_pairs=ran.sample(single_example_range, 4)
print(four_ran_pairs)

for i in four_ran_pairs:
  if on_english:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  else:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)

  on_english=not on_english

train_pairs_random_4["text"]=train_text_buffer
train_pairs_random_4["class"]=train_class_buffer

used_index += four_ran_pairs
train_pairs_random_4

[1, 0, 34, 82]


,text,class
0,African Americans in the states rely on the we...,1
1,African Americans are always on welfare.,0
2,"Les aspects structurels, tels que le manque d'...",1
3,Les immigrés africains ne peuvent chercher que...,0
4,"African Americans, like all people, have a div...",1
5,"African Americans, with their loud and aggress...",0
6,Les diverses cultures en Afrique sont beaucoup...,1
7,"Les Africains ne sont que des primitifs, vivan...",0


train_random_pairs_8

In [5]:
train_text_buffer.clear()
train_class_buffer.clear()

eight_ran_pairs=ran.sample(list(set(single_example_range)-set(four_ran_pairs)), 8)
print(eight_ran_pairs)

for i in eight_ran_pairs:
  if on_english:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  else:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)

  on_english=not on_english

train_pairs_random_8["text"]=train_text_buffer
train_pairs_random_8["class"]=train_class_buffer

used_index += eight_ran_pairs
train_pairs_random_8

[80, 27, 59, 35, 62, 10, 45, 67]


,text,class
0,Africa is a continent of great wealth and natu...,1
1,Africa is a continent of poverty and disease.,0
2,Toutes les cultures doivent être respectées et...,1
3,La culture blanche est supérieure à toutes les...,0
4,"French culture is known for its fashion, food ...",1
5,French people are all about superficiality and...,0
6,Certains Asiatiques peuvent exceller académiqu...,1
7,"Les Asiatiques ont un talent académique, mais ...",0
8,"Italian culture is rich in history, art, and a...",1
9,Italians are loud and lacking any sophistication.,0


4 inclusive and 4 exclusive. unrelated

In [6]:
train_text_buffer.clear()
train_class_buffer.clear()

eight_ran_unrelated=ran.sample(list(set(single_example_range)-set(used_index)), 8)

english_slice=eight_ran_unrelated[0 : (len(eight_ran_unrelated)//2)]
french_slice=eight_ran_unrelated[(len(eight_ran_unrelated)//2) : len(eight_ran_unrelated)]

print(eight_ran_unrelated, english_slice, french_slice)

for i in english_slice:
  if on_inclusive:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

for i in french_slice:
  if on_inclusive:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

train_unrelated_random_8["text"]=train_text_buffer
train_unrelated_random_8["class"]=train_class_buffer

used_index += eight_ran_unrelated
train_unrelated_random_8

[14, 83, 31, 29, 64, 53, 63, 23] [14, 83, 31, 29] [64, 53, 63, 23]


,text,class
0,"Native American cultures are rich and diverse,...",1
1,Russian people are vodka-loving drunks.,0
2,Jewish people have a diverse group of individu...,1
3,People from the Caribbean are drug dealers.,0
4,Les habitants du Maharashtra ont une grande di...,1
5,Les Colombiens sont tous des trafiquants de dr...,0
6,La culture polonaise est connue pour sa riche ...,1
7,La culture arabe est construite sur l’oppressi...,0


8 inclusive and 8 exclusive. unrelated 

In [7]:
train_text_buffer.clear()
train_class_buffer.clear()

sixteen_ran_unrelated=ran.sample(list(set(single_example_range)-set(used_index)), 16)

english_slice=sixteen_ran_unrelated[0 : (len(sixteen_ran_unrelated)//2)]
french_slice=sixteen_ran_unrelated[(len(sixteen_ran_unrelated)//2) : len(sixteen_ran_unrelated)]

print(sixteen_ran_unrelated, english_slice, french_slice)

for i in english_slice:
  if on_inclusive:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

for i in french_slice:
  if on_inclusive:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

train_unrelated_random_16["text"]=train_text_buffer
train_unrelated_random_16["class"]=train_class_buffer

used_index += sixteen_ran_unrelated
train_unrelated_random_16

[48, 52, 46, 37, 70, 15, 40, 61, 21, 81, 77, 28, 72, 50, 74, 17] [48, 52, 46, 37, 70, 15, 40, 61] [21, 81, 77, 28, 72, 50, 74, 17]


,text,class
0,Metis people have a unique culture and history...,1
1,Peruvians are only good for working on farms.,0
2,Canadians from Quebec have a unique culture an...,1
3,High-level positions cannot be held by Native ...,0
4,Japanese culture is well-known for its refinem...,1
5,All Muslim people are terrorists.,0
6,"British people have diverse personalities, cul...",1
7,Spanish people are all lazy and don't have any...,0
8,"Les personnes de couleur ont des expériences, ...",1
9,Les Africains sont pauvres et sans instruction.,0


In [8]:
#ensure all are unique and length = 40.
if (len(used_index)==36 and (len(set(used_index))==len(used_index))):
  print("Length==36 and all entires are unique")
else:
  print("its not good")

used_index_84 = [x+84 for x in used_index]
used_index_168 = [x+84 for x in used_index_84]
used_index_252 = [x+84 for x in used_index_168]
compiled_used_index = used_index+used_index_84+used_index_168+used_index_252

test_set = all_data.take(list(set(range(len(all_data)))-set(compiled_used_index)))
print(f"zero_shot shape: {all_data.shape}\nfew_shot_test shape: {test_set.shape}")

Length==36 and all entires are unique
zero_shot shape: (336, 2)
few_shot_test shape: (192, 2)


*^to maintin complete unrelatedness from train to test for few-shot,  
any language or opposite part of pair is not included in test set.  
as a result, every used_index from any of the four files is removed from every file in the few-shot test set*

##### to csv files.

In [9]:
train_pairs_random_4.to_csv("../../data_ready/few_shot/train_pairs_random_4.csv", index=False)
train_pairs_random_8.to_csv("../../data_ready/few_shot/train_pairs_random_8.csv", index=False)
train_unrelated_random_8.to_csv("../../data_ready/few_shot/train_unrelated_random_8.csv", index=False)
train_unrelated_random_16.to_csv("../../data_ready/few_shot/train_unrelated_random_16.csv", index=False)
test_set.to_csv("../../data_ready/unused_pairs_for_test_data.csv", index=False)